In [ ]:
import os
import random
from functools import reduce
import cv2
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
REBUILD_DATA = True
class CatOrDog():
    load_size = 128
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"

    LABELS = {CATS: 0, DOGS: 1}

    training_data = []

    cat_count = 0
    dog_count = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                try:
                    img = cv2.imread(os.path.join(label,f), 0)
                    img = cv2.resize(img, (self.load_size, self.load_size))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    if label == self.CATS:
                        self.cat_count += 1
                    else:
                        self.dog_count += 1

                except Exception as e:
                    pass
        np.random.shuffle(self.training_data)
        np.save('training_data.npy', self.training_data, allow_pickle=True)
        print("Cats:", self.cat_count)
        print("Dogs:", self.dog_count)

In [ ]:
def rebuild_data():
    cat_or_dog = CatOrDog()
    cat_or_dog.make_training_data()
    
def load_data():
    global trainset, testset, training_data
    TEST_SIZE = 0.1
    if REBUILD_DATA:
        rebuild_data()
    training_data = np.load('training_data.npy', allow_pickle=True)
    np.random.shuffle(training_data)
    train_idx = int(len(training_data)*(TEST_SIZE))
    trainset = training_data[:-train_idx]
    testset = training_data[-train_idx:]

load_data()

In [ ]:
IMG_SIZE = 100

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)
    
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(0.5),
    transforms.RandomAffine(360, (0.1,0.1), (0.9,1.1)),
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    AddGaussianNoise(0.05, 0.05),
])
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
])

In [ ]:
class CatOrDogDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx].copy()
        
        img[1] = torch.Tensor(img[1])
        if self.transform:
            img[0] = self.transform(img[0])
        return (img[0], img[1])
train_dataset = CatOrDogDataset(trainset, transform=train_transform)
test_dataset = CatOrDogDataset(testset, transform=train_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=100, num_workers=4, shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.conv4 = nn.Conv2d(128, 128, 3)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        
        x = torch.randn((IMG_SIZE, IMG_SIZE)).view(-1,1,IMG_SIZE, IMG_SIZE)
        fc1_size = self.convs(x).size()[1]
        
        self.fc1 = nn.Linear(fc1_size, 512)
        self.fc2 = nn.Linear(512, 32)
        self.fc3 = nn.Linear(32, 2)
        
    def convs(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = F.max_pool2d(x, (2,2))

        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = self.dropout1(x)
        x = F.max_pool2d(x, (2,2))

        x = self.conv3(x)
        x = F.relu(x)
        x = self.bn3(x)
        x = F.max_pool2d(x, (2,2))
        
        x = self.conv4(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = F.max_pool2d(x, (2,2))
        
        
        x = torch.flatten(x, 1)
        
        return x

    def forward(self, x):
        x = self.convs(x)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        
        x = F.softmax(x, 1)
        return x

In [ ]:
net = Net().to(device)
loss_function = nn.MSELoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
def train(epochs=5,model=None):
    if model == None:
        model = net
    net.train()
    correct = 0
    total = 0
    for epoch in range(epochs):
        losses = torch.Tensor([])
        bar = tqdm(train_loader)
        for data in bar:
            X, y = data
            X = X.to(device)
            y = y.to(device)
            real_classes = torch.argmax(y,1)
            net.zero_grad()
            output = net(X)
            predicted_classes = torch.argmax(output,1)
            loss = loss_function(output, y)
            losses = torch.cat((losses,torch.Tensor([loss])), 0)
            loss.backward()
            optimizer.step()
            correct += float(reduce(lambda x, y: x+1 if y else x, [a==b for a,b in zip(real_classes,predicted_classes)]))
            total += X.shape[0]
            bar.set_description_str("Epoch: "+str(epoch)+". Loss: "+str(round(float(losses.mean()),5))+". Acc: "+str(round(correct/total,3)))
    return float(losses.mean())

In [ ]:
def test(n=5,model=None):
    if model == None:
        model = net
    correct = 0
    total = 0
    losses = torch.Tensor([])
    net.eval()
    with torch.no_grad():
        bar = tqdm(range(n))
        for _ in bar:
            for data in test_loader:
                X, y = data
                X = X.to(device)
                y = y.to(device)
                real_classes = torch.argmax(y,1)
                
                net_out = model(X)
                
                loss = loss_function(net_out, y)
                losses = torch.cat((losses,torch.Tensor([loss])), 0)
                
                predicted_classes = torch.argmax(net_out,1)
                correct += float(reduce(lambda x, y: x+1 if y else x, [a==b for a,b in zip(real_classes,predicted_classes)]))
                
                total += X.shape[0]
            bar.set_description_str("Loss: "+str(round(float(losses.mean()),5))+". Acc: "+str(round(correct/total,3)))
    return round(correct/total,3)

In [ ]:
def save_checkpoint(model, path, loss, acc, optimizer):
    torch.save({
            'model': model,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'acc': acc,
            }, path)

In [ ]:
i = 0
while True:
    loss = train(3)
    acc = test(3)
    save_checkpoint(net,str(i)+'model.pt',loss, acc, optimizer)
    i+=1

In [ ]:
test(50,model=net)

In [ ]:
img = cv2.imread('caat.jpg',0)

test_img = test_transform(img)

net(test_img.view((-1,1,IMG_SIZE,IMG_SIZE)).to(device))[0]

In [ ]:
model = Net().to(device)
checkpoint = torch.load('checkpoint4.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()